In [1]:
KEY_ID = "PK34H595MF2B0I7M9Y9D"
SECRET_KEY = "o5Rv63Cqff6iIX0vTZGXVWjuN3c6uvXZXHvVGbmM"

In [3]:
import requests
import csv
import os

In [6]:
def fetch_intraday_stocks(stock, timeframe, start_date, end_date, output_file):
    """
        Fetch intraday stock data from Alpaca API and write it to a CSV file.
        Args:
            stock (str): The stock symbol.
            timeframe (str): The timeframe of the data.
            start_date (str): The start date of the data.
            end_date (str): The end date of the data.
            output_file (str): The output filepath.
    """
    url = f"https://data.alpaca.markets/v2/stocks/{stock}/bars"
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": KEY_ID,
        "APCA-API-SECRET-KEY": SECRET_KEY
    }

    params = {
        "timeframe": timeframe,
        "start": start_date,
        "end": end_date,
        "limit": 10000,
        "adjustment": "raw",
        "feed": "sip",
        "sort": "asc"
    }

    if not os.path.exists(output_file):
        with open(output_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["timestamp", "open", "high", "low", "close", "volume"])

    next_page_token = None

    while True:
        if next_page_token:
            params["page_token"] = next_page_token

        response = requests.get(url, headers=headers, params=params)

        if response.status_code != 200:
            print(f"Error: {response.status_code} - {response.text}")
            break

        data = response.json()

        if "bars" in data:
            with open(output_file, mode='a', newline='') as file:
                writer = csv.writer(file)
                for bar in data["bars"]:
                    writer.writerow([bar["t"], bar["o"], bar["h"], bar["l"], bar["c"], bar["v"]])

        next_page_token = data.get("next_page_token")

        if not next_page_token:
            break

In [21]:
fetch_intraday_stocks("SPY", "1Min", "2020-01-01", "2025-01-01", "./data/SPY_1min.csv")

In [42]:
def fetch_intraday_crypto(token, timeframe, start_date, end_date, output_file):
    """
        Fetch intraday crypto data from Alpaca API and write it to a CSV file.
        Args:
            token (str): The crypto token.
            timeframe (str): The timeframe of the data.
            start_date (str): The start date of the data.
            end_date (str): The end date of the data.
            output_file (str): The output filepath.
    """
    url = f"https://data.alpaca.markets/v1beta3/crypto/us/bars"
    headers = {
        "accept": "application/json"
    }

    params = {
        "symbols": token,
        "timeframe": timeframe,
        "start": start_date,
        "end": end_date,
        "limit": 10000,
        "sort": "asc"
    }


    # Create the output file if it doesn't exist
    if not os.path.exists(output_file):
        with open(output_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["timestamp", "open", "high", "low", "close", "volume", "number_of_trades", "vwap"])

    next_page_token = None

    # Iterate API requests until all data for a given period is fetched
    while True:
        if next_page_token:
            params["page_token"] = next_page_token
        response = requests.get(url, headers=headers, params=params)

        if response.status_code != 200:
            print(f"Error: {response.status_code} - {response.text}")
            break

        data = response.json()

        # Write data to the output file
        if "bars" in data:
            with open(output_file, mode='a', newline='') as file:
                writer = csv.writer(file)
                for bar in data["bars"][token]:
                    writer.writerow([
                        bar["t"],  # timestamp
                        bar["o"],  # open price
                        bar["h"],  # high price
                        bar["l"],  # low price
                        bar["c"],  # close price
                        bar["v"],  # volume
                        bar["n"],  # number of trades
                        bar["vw"]  # volume-weighted average price
                    ])

        next_page_token = data.get("next_page_token")

        if not next_page_token:
            break

In [46]:
fetch_intraday_crypto("ETH/USD", "5Min", "2020-01-01", "2025-01-01", "./data/ETH_5min.csv")